# LM Decoding

In [1]:
#@title Import libraries
import transformer_lens
import torch, gc, itertools, tqdm, scipy
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

import sys
sys.path.append('/home/jupyter/')
from paraMem.utils import helpers, modelHandlers, dataLoaders, decoding, gradient

In [2]:
model = modelHandlers.load_model(model_type="gpt-neo-125M", DEVICE="cpu")
modelHandlers.set_no_grad(model, ["embed", "pos_embed", "unembed"])

Loaded pretrained model gpt-neo-125M into HookedTransformer
set no_grad on ['embed', 'pos_embed', 'unembed']


In [ ]:
model.generate(input="An apple a day", max_new_tokens=10, stop_at_eos=True, eos_token_id=None, do_sample=False, top_k=None, top_p=None, temperature=1.0)

  0%|          | 0/10 [00:00<?, ?it/s]

"An apple a day keeps the doctor away.\n\nThat's the"

In [ ]:
model.generate(input="Jeff Dean was born on", max_new_tokens=10, stop_at_eos=True, eos_token_id=None, do_sample=True, top_k=None, top_p=None, temperature=1.0)

  0%|          | 0/10 [00:00<?, ?it/s]

'Jeff Dean was born on March 24, 1982 in Oranda, Kansas,'

In [1]:
def batch_decode(model, toks_NI:torch.LongTensor, new_toks:int=50, batch_size:int=5, stop_at_eos:bool=True):
    toks_NI = toks_NI.detach().to(model.cfg.device) ## detach and put on device
    preds_NI, trues_NI = torch.LongTensor(toks_NI.shape[0], new_toks), torch.LongTensor(toks_NI.shape[0], new_toks)
    
    toks_BNI = torch.split(toks_NI.detach(), batch_size) ## split in batches
    for b, toks_NI in tqdm.tqdm(enumerate(toks_BNI)):
        toks_NI_pref, toks_NI_true = toks_NI.detach()[...,:-new_toks], toks_NI.detach()[...,-new_toks:]
        toks_NI_pred = model.generate(input=toks_NI_pref, max_new_tokens=new_toks, stop_at_eos=stop_at_eos, eos_token_id=None, do_sample=False, top_k=None, top_p=None, temperature=1.0)
        toks_NI_pred = toks_NI_pred[...,-new_toks:] ## only take continuations  
        
        N = toks_NI.shape[0]
        current_B = toks_NI.shape[0]
        preds_NI[b*batch_size:(b*batch_size)+current_B,:] = toks_NI_pred
        trues_NI[b*batch_size:(b*batch_size)+current_B,:] = toks_NI_true
        del toks_NI_pred
    del toks_NI
    return preds_NI, trues_NI

NameError: name 'torch' is not defined